<a href="https://colab.research.google.com/github/walidlamhibi/tp4/blob/master/Mistral_7b_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

# Charger le fichier JSON
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
pip install sentence-transformers langchain transformers gradio torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install huggingface_hub

In [ ]:
from google.colab import userdata
from huggingface_hub import login # Import the login function from huggingface_hub

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.4 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [ ]:
# Importer les bibliothèques
import json
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login
from google.colab import userdata
import torch

# Authentification avec Hugging Face
HF_TOKEN = userdata.get('HF_TOKEN')  # Récupérer le token depuis les secrets de Colab
login(token=HF_TOKEN)

# Étape 1 : Charger le fichier JSON
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Étape 2 : Préparation des données (transformation en "chunks")
chunks = []

# À Propos + Aperçu
chunks.append(f"À propos: {data['à_propos']}\nAperçu: {data['aperçu']}")

# Services Salesforce
for service, desc in data['services'].items():
    chunks.append(f"Service {service}: {desc}")

# Expertise Principale
for categorie, details in data['expertise_principale'].items():
    if isinstance(details, dict):
        for sous_cat, val in details.items():
            chunks.append(f"Expertise {categorie} - {sous_cat}: {val}")
    else:
        chunks.append(f"Expertise {categorie}: {details}")

# Clients par secteur
for secteur, clients in data['projets_et_clients'].items():
    chunks.append(f"Clients {secteur}: {', '.join(clients)}")

# Récompenses
for recompense in data['réalisations_et_récompenses']:
    chunks.append(f"Récompense {recompense['titre']} ({recompense['année']}) à {recompense['lieu']}")

# Étape 3 : Embeddings et Indexation
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode(text).tolist()

embedding_model = SentenceTransformerEmbeddings('dangvantuan/sentence-camembert-base')
vector_db = FAISS.from_texts(chunks, embedding_model)
vector_db.save_local("gear9_index")

# Étape 4 : Chargement du Modèle LLM (Mistral-7B)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_llm = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=quant_config,
    device_map="auto",
    use_auth_token=True
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_auth_token=True)

# Étape 5 : Pipeline RAG
def format_prompt(context, question):
    return f"""Vous êtes un assistant de l'agence Gear9. Répondez de manière concise en français en utilisant uniquement le contexte suivant :

[CONTEXTE]
{context}
[/CONTEXTE]

Question : {question}
Réponse : """

def repondre(question):
    # Recherche dans la base vectorielle (k=1 pour limiter le contexte)
    docs = vector_db.similarity_search(question, k=1)
    context = "\n".join([doc.page_content for doc in docs])

    # Génération de la réponse
    inputs = tokenizer(format_prompt(context, question), return_tensors="pt").to("cuda")
    outputs = model_llm.generate(**inputs, max_new_tokens=100)
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_response.split("Réponse : ")[-1]  # Extraire uniquement la réponse finale

# Étape 6 : Interface Utilisateur (Gradio)
import gradio as gr

def chatbot(question, historique):
    reponse = repondre(question)
    return reponse

# Lancer l'interface
gr.ChatInterface(chatbot).launch()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.27k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c04c8c849bf0872225.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
